In [1]:
import numpy as np
import torch

In [5]:
gamma = torch.rand(5,5).uniform_(-1,1)
print(gamma)
gamma = gamma.sigmoid().gt_(torch.tensor(0.5))
gamma.diagonal().zero_()
print(gamma)
                            

tensor([[-0.1177,  0.2999,  0.8468,  0.8654,  0.4997],
        [-0.7504,  0.2826, -0.1647, -0.2045,  0.9564],
        [-0.6463, -0.2704, -0.8466,  0.9396,  0.7679],
        [ 0.6446, -0.8707,  0.0642, -0.2040,  0.9156],
        [ 0.4652, -0.7850,  0.9043,  0.3179, -0.5956]])
tensor([[0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 1.],
        [1., 0., 1., 1., 0.]])


In [46]:
# hist = [[0,1,2,0,3],[3,4,3,1,0],[0,3,2,1,4]]
hist = [[0,0,1,2,3],[0,3,4,3,1],[0,3,2,1,4]]
target = [4,2,4]
hist = torch.tensor(hist)
target = torch.tensor(target)

In [64]:
hist.nonzero().squeeze()

tensor([[0, 2],
        [0, 3],
        [0, 4],
        [1, 1],
        [1, 2],
        [1, 3],
        [1, 4],
        [2, 1],
        [2, 2],
        [2, 3],
        [2, 4]])

In [8]:
import numpy as np
a = ['hello']
np.array(a)

array(['hello'], dtype='<U5')

In [15]:
gamma[target].nonzero(as_tuple=False)

tensor([[0, 0],
        [0, 2],
        [0, 3],
        [1, 3],
        [1, 4],
        [2, 0],
        [2, 2],
        [2, 3]])

In [47]:
lengths = torch.tensor([3,4,4])
valid_his = hist.gt(0).long()
his_length = valid_his.sum(-1)
print(his_length)

sorted_his_length, sort_idx = torch.topk(his_length, k=len(lengths))

tensor([3, 4, 4])


In [21]:
iid_emb = torch.nn.Embedding(5, 4)

In [25]:
valid_his.unsqueeze(dim=-1).float().shape

torch.Size([3, 5, 1])

In [28]:
iid_emb(hist.abs()*valid_his).shape

torch.Size([3, 5, 4])

In [55]:
valid_his.unsqueeze(dim=-1).float()

tensor([[[0.],
         [0.],
         [1.],
         [1.],
         [1.]],

        [[0.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[0.],
         [1.],
         [1.],
         [1.],
         [1.]]])

In [48]:
his_vectors = iid_emb(hist.abs()*valid_his) * valid_his.unsqueeze(dim=-1).float()

In [39]:
his_vectors

tensor([[[ 0.0000,  0.0000, -0.0000,  0.0000],
         [-1.7279, -0.8900,  0.8295, -1.6214],
         [ 0.5841,  0.2310, -0.8600, -1.0928],
         [ 0.0000,  0.0000, -0.0000,  0.0000],
         [-0.0592,  0.4292,  0.0081, -0.8297]],

        [[-0.0592,  0.4292,  0.0081, -0.8297],
         [-0.8404,  0.2307, -0.5048,  1.9769],
         [-0.0592,  0.4292,  0.0081, -0.8297],
         [-1.7279, -0.8900,  0.8295, -1.6214],
         [ 0.0000,  0.0000, -0.0000,  0.0000]],

        [[ 0.0000,  0.0000, -0.0000,  0.0000],
         [-0.0592,  0.4292,  0.0081, -0.8297],
         [ 0.5841,  0.2310, -0.8600, -1.0928],
         [-1.7279, -0.8900,  0.8295, -1.6214],
         [-0.8404,  0.2307, -0.5048,  1.9769]]], grad_fn=<MulBackward0>)

In [49]:
sort_his_vector = his_vectors.index_select(dim=0, index=sort_idx)

In [53]:
sort_his_vector

tensor([[[ 0.0000,  0.0000, -0.0000,  0.0000],
         [-0.0592,  0.4292,  0.0081, -0.8297],
         [-0.8404,  0.2307, -0.5048,  1.9769],
         [-0.0592,  0.4292,  0.0081, -0.8297],
         [-1.7279, -0.8900,  0.8295, -1.6214]],

        [[ 0.0000,  0.0000, -0.0000,  0.0000],
         [-0.0592,  0.4292,  0.0081, -0.8297],
         [ 0.5841,  0.2310, -0.8600, -1.0928],
         [-1.7279, -0.8900,  0.8295, -1.6214],
         [-0.8404,  0.2307, -0.5048,  1.9769]],

        [[ 0.0000,  0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0000,  0.0000],
         [-1.7279, -0.8900,  0.8295, -1.6214],
         [ 0.5841,  0.2310, -0.8600, -1.0928],
         [-0.0592,  0.4292,  0.0081, -0.8297]]], grad_fn=<IndexSelectBackward>)

In [58]:
hist_packed = torch.nn.utils.rnn.pack_padded_sequence(hist, sorted_his_length, batch_first=True)
print(hist)
print(hist_packed)

tensor([[0, 0, 1, 2, 3],
        [0, 3, 4, 3, 1],
        [0, 3, 2, 1, 4]])
PackedSequence(data=tensor([0, 0, 0, 0, 3, 3, 1, 4, 2, 2, 3]), batch_sizes=tensor([3, 3, 3, 2]), sorted_indices=None, unsorted_indices=None)


In [50]:
history_packed = torch.nn.utils.rnn.pack_padded_sequence(sort_his_vector, sorted_his_length, batch_first=True)

In [51]:
history_packed

PackedSequence(data=tensor([[ 0.0000,  0.0000, -0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000,  0.0000],
        [-0.0592,  0.4292,  0.0081, -0.8297],
        [-0.0592,  0.4292,  0.0081, -0.8297],
        [ 0.0000,  0.0000, -0.0000,  0.0000],
        [-0.8404,  0.2307, -0.5048,  1.9769],
        [ 0.5841,  0.2310, -0.8600, -1.0928],
        [-1.7279, -0.8900,  0.8295, -1.6214],
        [-0.0592,  0.4292,  0.0081, -0.8297],
        [-1.7279, -0.8900,  0.8295, -1.6214]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([3, 3, 3, 2]), sorted_indices=None, unsorted_indices=None)

In [52]:
output, hidden = rnn(history_packed, None)
print(output)
print(hidden)

PackedSequence(data=tensor([[ 7.2568e-02,  1.9212e-01, -2.2542e-01,  3.6330e-03, -9.4501e-02,
          1.2062e-01],
        [ 7.2568e-02,  1.9212e-01, -2.2542e-01,  3.6330e-03, -9.4501e-02,
          1.2062e-01],
        [ 7.2568e-02,  1.9212e-01, -2.2542e-01,  3.6330e-03, -9.4501e-02,
          1.2062e-01],
        [ 3.1079e-01,  3.3411e-01, -4.2154e-01,  6.2677e-02,  5.3530e-02,
          1.9712e-01],
        [ 3.1079e-01,  3.3411e-01, -4.2154e-01,  6.2677e-02,  5.3530e-02,
          1.9712e-01],
        [ 7.5109e-02,  2.5640e-01, -3.3852e-01,  1.1056e-02, -1.0363e-01,
          1.9903e-01],
        [-2.5181e-01,  1.0953e-01, -3.7740e-01, -1.5275e-01, -1.0317e-01,
          2.4133e-01],
        [ 4.7527e-01,  2.5066e-01, -3.9029e-01,  2.0287e-01,  5.9163e-02,
          1.7335e-01],
        [ 1.6096e-01,  6.0185e-01, -5.6059e-01,  1.0657e-01,  3.7550e-01,
          4.2821e-01],
        [ 1.6965e-01,  2.8896e-01, -5.3561e-01, -5.5387e-04,  2.7547e-02,
          2.6741e-01],
        [ 

In [43]:
history_packed

PackedSequence(data=tensor([[-0.0592,  0.4292,  0.0081, -0.8297],
        [ 0.0000,  0.0000, -0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000,  0.0000],
        [-0.8404,  0.2307, -0.5048,  1.9769],
        [-0.0592,  0.4292,  0.0081, -0.8297],
        [-1.7279, -0.8900,  0.8295, -1.6214],
        [-0.0592,  0.4292,  0.0081, -0.8297],
        [ 0.5841,  0.2310, -0.8600, -1.0928],
        [ 0.5841,  0.2310, -0.8600, -1.0928],
        [-1.7279, -0.8900,  0.8295, -1.6214],
        [-1.7279, -0.8900,  0.8295, -1.6214]],
       grad_fn=<PackPaddedSequenceBackward>), batch_sizes=tensor([3, 3, 3, 2]), sorted_indices=None, unsorted_indices=None)

In [44]:
rnn = torch.nn.GRU(input_size=4, hidden_size=6, batch_first=True, num_layers=1)

In [45]:
output, hidden = rnn(history_packed, None)
print(output)
print(hidden)

PackedSequence(data=tensor([[ 0.3105,  0.2701, -0.2985,  0.0533,  0.0592,  0.1190],
        [ 0.0726,  0.1921, -0.2254,  0.0036, -0.0945,  0.1206],
        [ 0.0726,  0.1921, -0.2254,  0.0036, -0.0945,  0.1206],
        [-0.2538,  0.0928, -0.3261, -0.1568, -0.1066,  0.1877],
        [ 0.3108,  0.3341, -0.4215,  0.0627,  0.0535,  0.1971],
        [ 0.1721,  0.5767, -0.4910,  0.0957,  0.3675,  0.3859],
        [ 0.1754,  0.2845, -0.5088, -0.0075,  0.0216,  0.2310],
        [ 0.4753,  0.2507, -0.3903,  0.2029,  0.0592,  0.1734],
        [ 0.4068,  0.3360, -0.4384,  0.2269,  0.2359,  0.3062],
        [ 0.1834,  0.6234, -0.6567,  0.1115,  0.4139,  0.4562],
        [ 0.2837,  0.6297, -0.5619,  0.1969,  0.4409,  0.4396]],
       grad_fn=<CatBackward>), batch_sizes=tensor([3, 3, 3, 2]), sorted_indices=None, unsorted_indices=None)
tensor([[[ 0.1834,  0.6234, -0.6567,  0.1115,  0.4139,  0.4562],
         [ 0.2837,  0.6297, -0.5619,  0.1969,  0.4409,  0.4396],
         [ 0.4068,  0.3360, -0.4384,

In [10]:
import torch
a = [torch.tensor([]) for i in range(3)]

In [11]:
pad_his = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)

In [12]:
pad_his

tensor([], size=(3, 0))

In [71]:
gamma = torch.empty(5, 5)
gamma.diagonal().fill_(float("-inf"))
gammaexp = gamma.sigmoid().gt_(torch.tensor(0.5))
gammaexp.diagonal().zero_()
gammaexp.long()

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0]])

In [53]:
gammaexp

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0.]])

In [72]:
value, indice = torch.sort(gammaexp, dim=-1, descending=True)
indice

tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])

In [55]:
type(indice)

torch.Tensor